### 股票预测项目
本项目的目的是通过股票的历史行情价格来预测未来某只股票的涨跌。 问题本身其实就是二分类问题。数据是通过```tushare```库来获取到的，在压缩包里已经给出了一只股票的数据。本作业的目的是：
1. 根据已经给定的数据，构造出样本数据。在样本数据的构造过程我们需要使用特征工程，这个特征工程其实就是技术指标的提取。 
2. 提取完技术指标之后，做一些简单的数据处理
3. 构造训练数据和测试数据
4. 利用随机森林学习二分类器

本项目的重点是技术指标的提取，建议可以去看一下每一个技术指标是如何定义的。

预估项目完成时间： 2小时

In [40]:
# 导入相应的函数库
import pandas as pd
import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

# 技术指标的提取函数
from technical_indicators import *

In [3]:
# 导入股票数据，下面的股票数据是通过tushare库来获得的
stock = pd.read_csv("./600519.csv")
stock.head()

,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20
0,2019-09-12,1066.00,1109.98,1099.00,1066.00,41211.33,29.48,2.76,1114.276,1126.115,1108.345,40942.17,37385.21,37563.02
1,2019-09-11,1119.22,1119.97,1069.52,1068.00,81716.54,-54.33,-4.83,1123.276,1127.525,1105.640,39286.08,36197.12,36864.33
2,2019-09-10,1134.30,1135.00,1123.85,1120.01,26227.07,-12.67,-1.11,1134.374,1130.584,1104.330,29662.81,32726.43,34849.05
3,2019-09-09,1145.00,1148.00,1136.52,1135.00,29379.34,-5.97,-0.52,1137.604,1129.099,1099.035,30314.42,35320.15,35054.18
4,2019-09-06,1144.50,1146.15,1142.49,1131.00,26176.59,-1.51,-0.13,1138.052,1125.742,1093.141,30085.41,37232.00,37660.16


### 1. 对于股票数据提取技术指标
直接调用给定的技术指标库来获得这些数据， 但建议大家可以简单看一下这些指标是如何被计算出来的。 虽然没必要一定要掌握，但大致的计算逻辑可以学习一下的。 如果对某一种指标感兴趣，想深入理解建议在百度上搜索 ： “技术指标” + “指标名字”来获得相关的参考资料，比如搜索 “技术指标” + 'rate of change"， 有大量的资料可以参考的。

> ```TODO1```: 提取技术指标

In [4]:
# TODO: 提取各类技术指标， 你可以把所有的技术指标全部调用一遍，也可以选择几个来尝试。 或者感兴趣的话，可以把其他的技术指标也加进来。 
#       每个技术指标的参数是不一样的，但基本也就1-2个参数，最常用的参数是天数（函数里用n来表示）， 有些技术指标需要传入两个参数（比如MACD，
#       一个是针对于fast_line, 一个是针对于slow_line, 需要分别定义天数）。 由于每个指标都有参数，所以针对于同一类指标其实可以提取很多不同的特征的！

# 例子： stock = average_directional_movement_index(stock, 12, 26) #  提取技术指标并存放在新的dataframe中

stock = moving_average(stock, 5)  # 提取五天股价的moving_average
stock = exponential_moving_average(stock, 5)
stock = rate_of_change(stock, 5)  # 提取五天的变化率
stock = standard_deviation(stock, 5)

In [26]:
stock.head()

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,MA_5,EMA_5,ROC_5,STD_5,y
0,1066.00,1109.98,1099.00,1066.00,41211.33,29.48,2.76,1114.276,1126.115,1108.345,40942.17,37385.21,37563.02,682.881331,682.880907,-0.006290,12.320670,1.0
1,1119.22,1119.97,1069.52,1068.00,81716.54,-54.33,-4.83,1123.276,1127.525,1105.640,39286.08,36197.12,36864.33,682.881331,682.880907,-0.006290,12.320670,0.0
2,1134.30,1135.00,1123.85,1120.01,26227.07,-12.67,-1.11,1134.374,1130.584,1104.330,29662.81,32726.43,34849.05,682.881331,682.880907,-0.006290,12.320670,0.0
3,1145.00,1148.00,1136.52,1135.00,29379.34,-5.97,-0.52,1137.604,1129.099,1099.035,30314.42,35320.15,35054.18,682.881331,682.880907,-0.006290,12.320670,0.0
4,1144.50,1146.15,1142.49,1131.00,26176.59,-1.51,-0.13,1138.052,1125.742,1093.141,30085.41,37232.00,37660.16,1114.276000,1126.184123,0.039572,30.078824,0.0


### 2. 数据处理，以及训练样本和测试样本的提取
通过上面的环节我们已经提取好了所需要的技术指标。 接下来的环节是通过这批数据来构造训练数据和测试数据了。 
注：数据中存在着NAN，稍微思考一下为什么会出现这些NAN？ 为了去理解这些NAN的源头，需要看一下pandas里的rolling().mean()是如何工作的。在项目中，我们是通过历史一段时间的数据来预测未来的涨跌的，所以一定不能使用未来数据来预测未来，只能用历史数据来预测未来。 

> ```TODO2```： 做必要的数据预处理，并构建好样本数据。这里我们要预测的标签是第二天的涨跌。如果第二天的```close```价格 >  第一天的```close```价格，我们可以认为这个样本为正样本（1）， 如果价格小于第一天的```close```价格，就认为这个样本为负样本（0）。 构建完训练样本之后，在把样本通过```train_test_split```来划分为训练集和测试集。

In [11]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 18 columns):
date            614 non-null object
open            614 non-null float64
high            614 non-null float64
close           614 non-null float64
low             614 non-null float64
volume          614 non-null float64
price_change    614 non-null float64
p_change        614 non-null float64
ma5             614 non-null float64
ma10            614 non-null float64
ma20            614 non-null float64
v_ma5           614 non-null float64
v_ma10          614 non-null float64
v_ma20          614 non-null float64
MA_5            614 non-null float64
EMA_5           614 non-null float64
ROC_5           614 non-null float64
STD_5           614 non-null float64
dtypes: float64(17), object(1)
memory usage: 86.4+ KB


In [10]:
# 利用均值进行缺失值填充
for column in list(stock.columns[stock.isnull().sum() > 0]):
    mean_val = stock[column].mean()
    stock[column].fillna(mean_val, inplace=True)

In [22]:
# 构造标签项
for i in range(len(stock['close'])-1):
    if stock['close'][i] > stock['close'][i+1]:
        stock.loc[i,'y'] = 1
    else:
        stock.loc[i,'y'] = 0

In [25]:
stock.drop(columns=['date'], inplace=True, axis=1)

In [38]:
# TODO 2   完成样本数据的构造，并随机分成训练和测试数据
y = stock.loc[:,'y']
X = stock.iloc[:, :-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(460, 17) (460,) (154, 17) (154,)


### 3. 利用随机森林训练模型

> ```TODO3```：训练模型，通过交叉验证来训练。

In [42]:
# TODO: 训练随机森林模型，请尝试不同的参数，最后在测试集上输出最好的参数
params_grid = {
    'n_estimators':[200,300,400,500],
    'max_depth':[3,4,5]
}
rfc = RandomForestClassifier(n_estimators=400,criterion='entropy',
                             max_depth=5,min_samples_split=3,random_state=0)
clf_grid = GridSearchCV(rfc, params_grid, cv=5, scoring='f1')

clf_grid.fit(X_train,y_train)
print(clf_grid.best_params_)

{'max_depth': 3, 'n_estimators': 200}


In [43]:
# 利用所得最优参数重新训练随机森林
rfc = RandomForestClassifier(n_estimators=200,criterion='entropy',
                             max_depth=3,min_samples_split=3,random_state=0)
rfc.fit(X_train,y_train)
print(rfc.score(X_train,y_train), rfc.score(X_test, y_test))

0.9956521739130435 0.9935064935064936


拓展阅读： 从本项目中可以看到这里的核心其实就是一个一个指标，而且每一个指标都是通过大量的经验来构造出来的。 但有些复杂度的指标确实也比较难想出来。问题：有没有可能让计算机学出有用的指标呢？ 比如计算机可以学出这样的指标 = (close- open) * volum - close * close - open   虽然这个指标有点看不懂，但有可能是有效的，有没有可能让AI做这件事情？？？ 如果对这些感兴趣，可以参考一下下方链接： https://www.baidu.com/link?url=WmpaRS35js8T8gAUzaF6_rvdepe0OqpgmeU0fTxhXzMZnKCUXIECQeUFB6VTpFjg&wd=&eqid=b04a03b600117ba2000000035d88bea9